In [ ]:
# read shp files
# rename columns 
# merge into one 
# merge into the datafile



In [21]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [22]:
# clean up: 

def dropstuff(df):
    df = df.drop(['DN', 'glaceir', 'layer', 'path', 'Name', 'layer_2', 'path_2', 'area_m', 
             'perimeter', 'glaciersum', 'geometry'], axis = 1)
    return df

In [23]:
# read all .shp files

folder = '/Users/varyabazilova/Desktop/alluvial_fans/dem_all/all_points_so_far/10_datawrangl_formodel!!/shapes/permafrost/'

permafrost4 = pd.DataFrame(gpd.read_file(folder + 'overlap_permafrost4.shp')).rename(columns={"permafrost": "permafrost4", "permafro_1": "permafrost4_frac"})
permafrost3 = pd.DataFrame(gpd.read_file(folder + 'overlap_permafrost3.shp')).rename(columns={"permafrost": "permafrost3", "permafro_1": "permafrost3_frac"})
permafrost2 = pd.DataFrame(gpd.read_file(folder + 'overlap_permafrost2.shp')).rename(columns={"permafrost": "permafrost2", "permafro_1": "permafrost2_frac"})
permafrost1 = pd.DataFrame(gpd.read_file(folder + 'overlap_permafrost1.shp')).rename(columns={"permafrost": "permafrost1", "permafro_1": "permafrost1_frac"})

In [24]:
# clean up 
permafrost4 = dropstuff(permafrost4)
permafrost3 = dropstuff(permafrost3)
permafrost2 = dropstuff(permafrost2)
permafrost1 = dropstuff(permafrost1)

In [25]:
# merge into the dfclim data 

folder = '/Users/varyabazilova/Desktop/alluvial_fans/dem_all/all_points_so_far/ml_model/data_current/'

dfclim = pd.read_csv(folder + 'dfclim_3aug_rainydays.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)

In [26]:
dfclim1 = dfclim.merge(permafrost1, on =['x_centroid', 'y_centroid'], how = 'left')
dfclim1 = dfclim1.merge(permafrost2, on =['x_centroid', 'y_centroid'], how = 'left')
dfclim1 = dfclim1.merge(permafrost3, on =['x_centroid', 'y_centroid'], how = 'left')
dfclim1 = dfclim1.merge(permafrost4, on =['x_centroid', 'y_centroid'], how = 'left')

In [27]:
dfclim1 = dfclim1.drop_duplicates()

# dfclim1.to_csv('out/dfclim_permafrost9aug.csv')

In [28]:
dfclim1.head()
dfclim1.columns

Index(['DN', 'target', 'Name', 'x_centroid', 'y_centroid', 'area_m',
       'perimeter', 'glaciersum', 'mean_elevation', 'median_elevation',
       'std_elevation', 'min_elevation', 'max_elevation', 'range_elevation',
       'variance_elevation', 'region', 'region_name', 'mean_slope',
       'median_slope', 'std_slope', 'min_slope', 'max_slope', 'range_slope',
       'variance_slope', 'glarea_percent', 'glacier', 'x_wgs', 'y_wgs', 'M',
       'Rc', 'Cc', 'mean_annual_T', 'mean_jan_T', 'mean_july_T',
       'mean_monsoon_T', 'mean_no_monsoon_T', 'count_zeros', 'belowzero_frac',
       'avgtemp_belowzero', 'mean_annual_tp', 'mean_annualsum_tp',
       'mean_daylymonsoon_tp_x', 'mean_daylymonsoon_tp_y',
       'mean_monsoon_sum_tp', 'monsoon_tp_frac', 'n_rainy_days',
       'rainy_days_frac', 'permafrost1', 'permafrost1_frac', 'permafrost2',
       'permafrost2_frac', 'permafrost3', 'permafrost3_frac', 'permafrost4',
       'permafrost4_frac'],
      dtype='object')

### continious permafrost (type4) yes/no 

In [29]:
# pd.Series(np.where(sample.housing.values == 'yes', 1, 0),

dfclim1['cont_permafrost'] = np.where(dfclim1.permafrost4 > 0, 1, 0)
dfclim1['cont_permafrost_frac50'] = np.where(dfclim1.permafrost4_frac > 50, 1, 0) 

In [30]:
# make a column about all types of permafrost (like sum them or smth) 
# then look for where it is not 0 -> where(condition) 
# then remove the column 

dfclim1['all_permafrost_frac'] = dfclim1.permafrost1_frac + dfclim1.permafrost2_frac + dfclim1.permafrost3_frac + dfclim1.permafrost4_frac
dfclim1['any_permafrost'] = np.where(dfclim1.all_permafrost_frac > 0, 1, 0)


In [13]:
# dfclim1.permafrost4.values

In [31]:
dfclim1.columns

Index(['DN', 'target', 'Name', 'x_centroid', 'y_centroid', 'area_m',
       'perimeter', 'glaciersum', 'mean_elevation', 'median_elevation',
       'std_elevation', 'min_elevation', 'max_elevation', 'range_elevation',
       'variance_elevation', 'region', 'region_name', 'mean_slope',
       'median_slope', 'std_slope', 'min_slope', 'max_slope', 'range_slope',
       'variance_slope', 'glarea_percent', 'glacier', 'x_wgs', 'y_wgs', 'M',
       'Rc', 'Cc', 'mean_annual_T', 'mean_jan_T', 'mean_july_T',
       'mean_monsoon_T', 'mean_no_monsoon_T', 'count_zeros', 'belowzero_frac',
       'avgtemp_belowzero', 'mean_annual_tp', 'mean_annualsum_tp',
       'mean_daylymonsoon_tp_x', 'mean_daylymonsoon_tp_y',
       'mean_monsoon_sum_tp', 'monsoon_tp_frac', 'n_rainy_days',
       'rainy_days_frac', 'permafrost1', 'permafrost1_frac', 'permafrost2',
       'permafrost2_frac', 'permafrost3', 'permafrost3_frac', 'permafrost4',
       'permafrost4_frac', 'cont_permafrost', 'cont_permafrost_frac50',


In [32]:
# dfclim1.to_csv('out/dfclim_permafrost9aug.csv')

In [33]:
#  cont_permafrost - 1/0 - if there is any continious permafrost in the catchment
#  any_permafrost - 1/0 - if there is any permafrost in the catchment 
#  all_permafrost_frac - whats the total fraction of the area covered by any type of permafrost 
#  cont_permafrost_frac50 - 1/0 - if more than 50% of the area are covered by continious permafrost 

# permafrost4 - continious permafrost (>90% coverage)
# permafrost3 - discontinious permafrost (50-90%)
# permafrost2 - sporadic permafrost (10-50%)
# permafrost1 - isolated patches (<10%) 

# permafrost4_frac - fraction of area with continious permafrost (>90% coverage)
# permafrost3_frac - fraction of area with discontinious permafrost (50-90%)
# permafrost2_frac - fraction of area with sporadic permafrost (10-50%)
# permafrost1_frac - fraction of area with isolated patches (<10%) 

